In [ ]:
# %pip install yfinance
# %pip install scipy
# %pip install matplotlib
# %pip install pyfolio
# %pip uninstall pyfolio
# %pip install git+https://github.com/quantopian/pyfolio
# %pip install pandas
# %pip install pandas-market-calendars

In [1]:
# Ignore printing all warnings
import warnings
warnings.filterwarnings('ignore')

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pyfolio as pf
import datetime as dt
import pandas_datareader.data as web
import os
# import warnings

# print all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.stats import rankdata

In [2]:
ticker_list_canary = ['SPY', 'EEM', 'EFA', 'AGG']
ticker_list_defensive = ['TIP', 'DBC', 'BIL', 'IEF', 'TLT', 'LQD', 'AGG']
ticker_list_aggressive = ['QQQ', 'EEM', 'EFA', 'AGG']
ticker_list_balanced = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'DBC', 'GLD', 'TLT', 'HYG', 'LQD']
rebalance_unit = 'Month'
rebalance_freq = 1
rebalance_shift = 0
skipped_period = 0
no_played_ETFs = {'offAgg' : 1, 'offBal' : 6, 'def' : 3}
abs_threshold = -1
start_date = '2004-01-01'
end_date = '2022-10-24'

In [3]:
ticker_list_all = ticker_list_canary + ticker_list_defensive + ticker_list_aggressive + ticker_list_balanced
adj_close_price = yf.download(ticker_list_all,start = pd.to_datetime(start_date) + pd.DateOffset(years= -2),end = pd.to_datetime(end_date) + pd.DateOffset(days= 1) )['Adj Close']

In [4]:
df = adj_close_price.copy()
df['Year'], df['Month'], df['Week'] = df.index.year, df.index.month, df.index.isocalendar().week
df['Monthly_Rb'] = df.Month != df.Month.shift(-1)
df['Weekly_Rb'] = df.Week != df.Week.shift(-1)
df['Daily_Rb'] = True
df['RebalanceUnit'] = df.Monthly_Rb if rebalance_unit == 'Month' else df.Weekly_Rb if rebalance_unit == 'Week' else df.Daily_Rb
df.RebalanceUnit = df.RebalanceUnit.shift(rebalance_shift).fillna(False) 
df.RebalanceUnit[0] = True
df['NoPeriod'] = df.RebalanceUnit.shift(1).cumsum()
df.NoPeriod[0] = 0
df['NoData'] = np.arange(len(df)) + 1
df['Rebalance'] = np.where((df.RebalanceUnit == True) & (df.NoPeriod.mod(rebalance_freq) == 0), True, False)

In [ ]:
dailyret = adj_close_price/adj_close_price.shift(1) - 1

In [ ]:
canary_lbs_base = np.array([1, 3, 6, 12])
canary_lbs = canary_lbs_base * 21
canary_weights = [12, 4, 2, 1]
canary_df = df[ticker_list_canary]
canary_rets = (canary_df / canary_df.shift(canary_lbs[0]) - 1) * canary_weights[0] + (canary_df / canary_df.shift(canary_lbs[1]) - 1) * canary_weights[1] + (canary_df / canary_df.shift(canary_lbs[2]) - 1) * canary_weights[2] + (canary_df / canary_df.shift(canary_lbs[3]) - 1) * canary_weights[3]
canary_signal = (canary_rets > 0).sum(1) / len(ticker_list_canary)

In [ ]:
rel_mom_lbs_base = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
rel_mom_lbs = (rel_mom_lbs_base + skipped_period) * 21
rel_mom_weights = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
defensive_df = df[ticker_list_defensive]
aggressive_df = df[ticker_list_aggressive]
balanced_df = df[ticker_list_balanced]


In [ ]:
def rel_mom_weighted(used_df_p, rel_mom_lbs_p, rel_mom_weights_p, skipped_period_p):
    used_avg_price = pd.DataFrame(0, index = used_df_p.index, columns = used_df_p.columns)
    for i in range(len(rel_mom_lbs_p)):
        used_avg_price += used_df_p.shift(rel_mom_lbs_p[i]) * rel_mom_weights_p[i]
    used_avg_price = used_avg_price / sum(rel_mom_weights_p)
    used_rets = used_df_p.shift(skipped_period_p * 21) / used_avg_price - 1
    return used_rets


In [ ]:
defensive_rets = rel_mom_weighted(defensive_df, rel_mom_lbs, rel_mom_weights, skipped_period)
aggressive_rets = rel_mom_weighted(aggressive_df, rel_mom_lbs, rel_mom_weights, skipped_period)
balanced_rets = rel_mom_weighted(balanced_df, rel_mom_lbs, rel_mom_weights, skipped_period)
defensive_rank = defensive_rets.rank(axis = 1, ascending = False)
aggressive_rank = aggressive_rets.rank(axis = 1, ascending = False)
balanced_rank = balanced_rets.rank(axis = 1, ascending = False)
defensive_played = ((defensive_rets > abs_threshold) & (defensive_rank <= no_played_ETFs['def'])).sum(1)
aggressive_played = ((aggressive_rets > abs_threshold) & (aggressive_rank <= no_played_ETFs['offAgg'])).sum(1)
balanced_played = ((balanced_rets > abs_threshold) & (balanced_rank <= no_played_ETFs['offBal'])).sum(1)
defensive_weights = (defensive_rank.divide(defensive_rank, axis = 0).divide(defensive_played, axis =0)).where((defensive_rank <= no_played_ETFs['def']) & (defensive_rets > abs_threshold), 0)
aggressive_weights = (aggressive_rank.divide(aggressive_rank, axis = 0).divide(aggressive_played, axis =0)).where((aggressive_rank <= no_played_ETFs['offAgg']) & (aggressive_rets > abs_threshold), 0)
balanced_weights = (balanced_rank.divide(balanced_rank, axis = 0).divide(balanced_played, axis =0)).where((balanced_rank <= no_played_ETFs['offBal']) & (balanced_rets > abs_threshold), 0)
def_cash_weight = 1 - defensive_weights.sum(axis = 1)
agg_cash_weight = 1 - aggressive_weights.sum(axis = 1)
bal_cash_weight = 1 - balanced_weights.sum(axis = 1)

In [ ]:
no_agg_etfs = len(ticker_list_aggressive)
no_bal_etfs = len(ticker_list_balanced)
no_def_etfs = len(ticker_list_defensive)
canary_signal = np.array(canary_signal)
aggressive_weights_helper = np.array(aggressive_weights)
balanced_weights_helper = np.array(balanced_weights)
defensive_weights_helper = np.array(defensive_weights)

agg_def_df = pd.concat([aggressive_df, defensive_df], axis = 1)
no_rows, no_cols = agg_def_df.shape
agg_def_weights_helper = np.zeros((no_rows, no_cols))
for i in range(no_rows):
    if canary_signal[i] == 1:
        for j in range(no_agg_etfs):
            agg_def_weights_helper[i, j] = aggressive_weights_helper[i, j]
    else:
        for j in range(no_def_etfs):
            agg_def_weights_helper[i, j + no_agg_etfs] = defensive_weights_helper[i, j]
agg_def_weights = pd.DataFrame(agg_def_weights_helper, index = agg_def_df.index, columns = agg_def_df.columns)
agg_def_cash_weight = 1 - agg_def_weights.sum(axis = 1)

bal_def_df = pd.concat([balanced_df, defensive_df], axis = 1)
no_rows, no_cols = bal_def_df.shape
bal_def_weights_helper = np.zeros((no_rows, no_cols))
for i in range(no_rows):
    if canary_signal[i] == 1:
        for j in range(no_bal_etfs):
            bal_def_weights_helper[i, j] = balanced_weights_helper[i, j]
    else:
        for j in range(no_def_etfs):
            bal_def_weights_helper[i, j + no_bal_etfs] = defensive_weights_helper[i, j]
bal_def_weights = pd.DataFrame(bal_def_weights_helper, index = bal_def_df.index, columns = bal_def_df.columns)
bal_def_cash_weight = 1 - bal_def_weights.sum(axis = 1)


In [ ]:
def positions_pv(acp_p, rebalance_p, weights_p, cash_weight_p, start_date_p):
    adjclose = acp_p.fillna(1).to_numpy()
    rebalanceday = rebalance_p.to_numpy()
    weights = weights_p.fillna(0).to_numpy()
    cash_weight = cash_weight_p.to_numpy()
    no_rows, no_cols = acp_p.shape

    positions = np.zeros((no_rows, no_cols))
    pv = np.ones((no_rows,1))
    cash = np.ones((no_rows,1))
    start_ind = np.argmax(acp_p.index >= start_date_p)
    for i in range(start_ind, no_rows):
        pv[i] = 0
        for j in range(no_cols):
            if rebalanceday[i-1] == True:
                positions[i,j] = pv[i-1] * weights[i-1,j] / adjclose[i-1,j]                
            else:
                positions[i,j] = positions[i-1,j]
            pv[i] += positions[i,j] * adjclose[i,j]
        if rebalanceday[i-1] == True:
            cash[i] = pv[i-1] * cash_weight[i-1]
        else:
            cash[i] = cash[i-1]
        pv[i] += cash[i] 
    
    positions_df = pd.DataFrame(positions, index = acp_p.index, columns = acp_p.columns)
    positions_df_sel = positions_df.iloc[start_ind-1:no_rows,:]
    pv_df =  pd.DataFrame(pv, index = acp_p.index)
    pv_df_sel = pv_df.iloc[start_ind-1:no_rows,0]
    cash_df =  pd.DataFrame(cash, index = acp_p.index)
    cash_df_sel = cash_df.iloc[start_ind-1:no_rows,0]
    return positions_df_sel, cash_df_sel, pv_df_sel

In [ ]:
def positions_pv_ew(acp_p, rebalance_p, start_date_p):
    adjclose = acp_p.fillna(0).to_numpy()
    rebalanceday = rebalance_p.to_numpy()
    no_rows, no_cols = acp_p.shape
    no_ava_etfs = (acp_p > 0).sum(1)

    positions = np.zeros((no_rows, no_cols))
    pv = np.ones((no_rows,1))
    cash = np.ones((no_rows,1))
    start_ind = np.argmax(acp_p.index >= start_date_p)

    pv[start_ind] = 0
    for j in range(no_cols):
        positions[start_ind, j] = pv[start_ind - 1] * (1 / no_ava_etfs[start_ind - 1]) / adjclose[start_ind - 1, j] if adjclose[start_ind - 1, j] > 0 else 0
        pv[start_ind] += positions[start_ind, j] * adjclose[start_ind, j]
    
    for i in range(start_ind + 1, no_rows):
        pv[i] = 0
        for j in range(no_cols):
            if rebalanceday[i-1] == True:
                positions[i,j] = pv[i-1] * (1 / no_ava_etfs[i-1]) / adjclose[i-1,j] if adjclose[i-1, j] > 0 else 0                
            else:
                positions[i,j] = positions[i-1,j]
            pv[i] += positions[i,j] * adjclose[i,j]
        if rebalanceday[i-1] == True:
            cash[i] = 0
        else:
            cash[i] = 0
        pv[i] += cash[i] 
    
    positions_df = pd.DataFrame(positions, index = acp_p.index, columns = acp_p.columns)
    positions_df_sel = positions_df.iloc[start_ind-1:no_rows,:]
    pv_df =  pd.DataFrame(pv, index = acp_p.index)
    pv_df_sel = pv_df.iloc[start_ind-1:no_rows,0]
    cash_df =  pd.DataFrame(cash, index = acp_p.index)
    cash_df_sel = cash_df.iloc[start_ind-1:no_rows,0]
    return positions_df_sel, cash_df_sel, pv_df_sel

In [ ]:
pos_agg, cash_agg, pv_agg = positions_pv(aggressive_df, df.Rebalance, aggressive_weights, agg_cash_weight, start_date)
pos_bal, cash_bal, pv_bal = positions_pv(balanced_df, df.Rebalance, balanced_weights, bal_cash_weight, start_date)
pos_def, cash_def, pv_def = positions_pv(defensive_df, df.Rebalance, defensive_weights, def_cash_weight, start_date)
pos_agg_def, cash_agg_def, pv_agg_def = positions_pv(agg_def_df, df.Rebalance, agg_def_weights, agg_def_cash_weight, start_date)
pos_bal_def, cash_bal_def, pv_bal_def = positions_pv(bal_def_df, df.Rebalance, bal_def_weights, bal_def_cash_weight, start_date)
pos_ew, cash_ew, pv_ew = positions_pv_ew(adj_close_price, df.Rebalance, start_date)

ew_rets = pv_ew / pv_ew.shift(1) - 1
agg_rets = pv_agg / pv_agg.shift(1) - 1
bal_rets = pv_bal / pv_bal.shift(1) - 1
def_rets = pv_def / pv_def.shift(1) - 1
agg_def_rets = pv_agg_def / pv_agg_def.shift(1) - 1
bal_def_rets = pv_bal_def / pv_bal_def.shift(1) - 1
aggressive_weights2 = aggressive_weights.copy()
aggressive_weights2['cash'] = agg_cash_weight
balanced_weights2 = balanced_weights.copy()
balanced_weights2['cash'] = bal_cash_weight
defensive_weights2 = defensive_weights.copy()
defensive_weights2['cash'] = def_cash_weight
agg_def_weights2 = agg_def_weights.copy()
agg_def_weights2['cash'] = agg_def_cash_weight
bal_def_weights2 = bal_def_weights.copy()
bal_def_weights2['cash'] = bal_def_cash_weight


In [ ]:
# with open('csv_data4.csv', 'w') as csv_file:
#     pos_bal_def.to_csv(path_or_buf=csv_file, sep = ';')

In [ ]:
# np.savetxt('out.csv', canary_signal, delimiter=';')

In [ ]:
pf.create_full_tear_sheet(bal_def_rets, positions = bal_def_weights2, benchmark_rets = ew_rets)